!pip install yfinance pandas numpy matplotlib

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dowloading the Stock Data

In [11]:
#getting the real srock prices for calc rsi and build the startegy
data = yf.download('BAC' , start = '2020-01-01' , end = '2021-01-01')
data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,BAC,BAC,BAC,BAC,BAC
Date,,,,,
2020-01-02,31.275545,31.293096,30.968407,31.021058,37614200
2020-01-03,30.626162,30.845547,30.503303,30.696363,50357900
2020-01-06,30.582281,30.626161,30.161062,30.196165,42185000
2020-01-07,30.380453,30.643715,30.301474,30.450658,34149000
2020-01-08,30.687590,30.880646,30.319021,30.336572,45311600


# Calculating the RSI

In [35]:
# function to calculate the RSI
def calc_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss 
    return 100 - (100 / (1 + rs))

# apply it to the correct column
data['RSI'] = calc_rsi(data['Close'])

 # Creating Buy/Sell Signals 

In [37]:
data['Signal'] = 0 
data.loc[data['RSI']< 30 , 'Signal']  = 1
data.loc[data['RSI']> 70 , 'Signal']  = -1

data['Position'] = data['Signal'].diff()

# Plotting thr RSI strategy

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(data['Close'], label='Close Price', alpha=0.5)
plt.scatter(data.index[data['Position'] == 1],
            data['Close'][data['Position'] == 1],
            label='Buy-Signal', marker='^', color='green', lw=3)
plt.scatter(data.index[data['Position'] == -1],
            data['Close'][data['Position'] == -1],
            label='Sell-Signal', marker='v', color='red', lw=3)  
plt.title('RSI STRAT FOR TRADING(BUY below 30  &  SELL below 70)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid()
plt.show()